In [1]:
import os
from pyspark.sql.types import *
from pyspark.sql import functions as F
from pyspark.sql import SQLContext
import sys
import itertools

from math import sqrt
from operator import add
from os.path import join, isfile, dirname

from pyspark import SparkConf, SparkContext
from pyspark.mllib.recommendation import ALS
from pyspark.mllib.evaluation import RegressionMetrics
from pyspark.mllib.regression import LabeledPoint
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

from pyspark.sql import Row

print sc

In [2]:
plays_df_schema = StructType(
  [StructField('userId', StringType()),
   StructField('songId', StringType()),
   StructField('Plays', IntegerType())]
)

In [3]:
raw_plays_df = sqlContext.read.format('com.databricks.spark.csv') \
                              .options(delimiter = '\t', header=True,inferSchema=False) \
                              .schema(plays_df_schema) \
                              .load("/Users/samanthapace/Desktop/SongData/train_triplets.txt")

In [4]:
# change ids from strings to integers .F.shiftRightUnsigned()
userId_change = raw_plays_df.select('userId').distinct().select('userId', F.monotonically_increasing_id().alias('new_userId'))

songId_change = raw_plays_df.select('songId').distinct().select('songId', F.monotonically_increasing_id().alias('new_songId'))

#userId_change.select('new_userId', ((userId_change['new_userId'])/100).alias("use_userId")).show()




In [ ]:
unique_users = userId_change.count()
unique_songs = songId_change.count()
print('Number of unique users: {0}'.format(unique_users))
print('Number of unique songs: {0}'.format(unique_songs))

# join dataframes
raw_plays_df_with_int_ids = raw_plays_df.join(userId_change, 'userId')
#.join(songId_change, 'songId')

# remove half users to make more manageable
#raw_plays_df_with_int_ids = raw_plays_df_with_int_ids.filter(raw_plays_df_with_int_ids.new_userId < unique_users/2)


# cache
raw_plays_df_with_int_ids.cache()
raw_plays_df_with_int_ids.show(5)


In [105]:
song_ids_with_total_listens = raw_plays_df_with_int_ids.groupBy('songId') \
                                                       .agg(F.count(raw_plays_df_with_int_ids.Plays).alias('User_Count'),
                                                            F.sum(raw_plays_df_with_int_ids.Plays).alias('Total_Plays')) \
                                                       .orderBy('Total_Plays', ascending = False)

print 'song_ids_with_total_listens:'
song_ids_with_total_listens.show(3, truncate=False)


song_ids_with_total_listens:
+------------------+----------+-----------+
|songId            |User_Count|Total_Plays|
+------------------+----------+-----------+
|SOBSUJE12A6D4F8CF5|1         |2          |
|SOBBMDR12A8C13253B|1         |2          |
|SOBFNSP12AF72A0E22|1         |1          |
+------------------+----------+-----------+
only showing top 3 rows



In [108]:
# We'll hold out 60% for training, 20% of our data for validation, and leave 20% for testing
seed = 1800009193L
(split_60_df, split_a_20_df, split_b_20_df) = raw_plays_df_with_int_ids.randomSplit([0.6, 0.2, 0.2], seed = seed)

# Let's cache these datasets for performance
training_df = split_60_df.cache()
validation_df = split_a_20_df.cache()
test_df = split_b_20_df.cache()

print('Training: {0}, validation: {1}, test: {2}\n'.format(
  training_df.count(), validation_df.count(), test_df.count())
)
training_df.show(3)
validation_df.show(3)
test_df.show(3)

#Number of plays needs to be double type, not integers
validation_df = validation_df.withColumn("Plays", validation_df["Plays"].cast(DoubleType()))
validation_df.show(10)





Training: 6, validation: 4, test: 2

+------------------+--------------------+-----+------------+-------------+
|            songId|              userId|Plays|  new_userId|   new_songId|
+------------------+--------------------+-----+------------+-------------+
|SOAPDEY12A81C210A9|b80344d063b5ccb32...|    1|240518168576| 609885356032|
|SOBBMDR12A8C13253B|b80344d063b5ccb32...|    2|240518168576|1537598291968|
|SOBFOVM12A58A7D494|b80344d063b5ccb32...|    1|240518168576| 747324309504|
+------------------+--------------------+-----+------------+-------------+
only showing top 3 rows

+------------------+--------------------+-----+------------+-------------+
|            songId|              userId|Plays|  new_userId|   new_songId|
+------------------+--------------------+-----+------------+-------------+
|SOBFNSP12AF72A0E22|b80344d063b5ccb32...|    1|240518168576| 506806140928|
|SOBXHDL12A81C204C0|b80344d063b5ccb32...|    1|240518168576|1305670057984|
|SOCNMUH12A6D4F6E6D|b80344d063b5ccb32.

In [123]:
# Let's initialize our ALS learner
als = ALS()

# Now set the parameters for the method
als.setMaxIter(5)\
   .setSeed(seed)\
   .setItemCol("new_songId")\
   .setRatingCol("Plays")\
   .setUserCol("new_userId")

# Now let's compute an evaluation metric for our test dataset
# We Create an RMSE evaluator using the label and predicted columns
reg_eval = RegressionEvaluator(predictionCol="prediction", labelCol="Plays", metricName="rmse")
tolerance = 0.03
ranks = [4, 8, 12, 16]
regParams = [0.15, 0.2, 0.25]
errors = [[0]*len(ranks)]*len(regParams)
models = [[0]*len(ranks)]*len(regParams)
err = 0
min_error = float('inf')
best_rank = -1
i = 0


for regParam in regParams:
    j = 0
    for rank in ranks:
        # Set the rank here:
        als.setParams(rank = rank, regParam = regParam)
        # Create the model with these parameters.
        training_df.show()
        model = als.fit(training_df)
        # Run the model to create a prediction. Predict against the validation_df.
        predict_df = model.transform(validation_df)
        # Remove NaN values from prediction (due to SPARK-14489)
        predicted_plays_df = predict_df.filter(predict_df.prediction != float('nan'))
        predicted_plays_df = predicted_plays_df.withColumn("prediction", F.abs(F.round(predicted_plays_df["prediction"],0)))
        # Run the previously created RMSE evaluator, reg_eval, on the predicted_ratings_df DataFrame
        error = reg_eval.evaluate(predicted_plays_df)
        errors[i][j] = error
        models[i][j] = model
        print 'For rank %s, regularization parameter %s the RMSE is %s' % (rank, regParam, error)
        if error < min_error:
            min_error = error
            best_params = [i,j]
        j += 1
    i += 1
    
als.setRegParam(regParams[best_params[0]])
als.setRank(ranks[best_params[1]])
print 'The best model was trained with regularization parameter %s' % regParams[best_params[0]]
print 'The best model was trained with rank %s' % ranks[best_params[1]]
my_model = models[best_params[0]][best_params[1]]



+------------------+--------------------+-----+------------+-------------+
|            songId|              userId|Plays|  new_userId|   new_songId|
+------------------+--------------------+-----+------------+-------------+
|SOAPDEY12A81C210A9|b80344d063b5ccb32...|    1|240518168576| 609885356032|
|SOBBMDR12A8C13253B|b80344d063b5ccb32...|    2|240518168576|1537598291968|
|SOBFOVM12A58A7D494|b80344d063b5ccb32...|    1|240518168576| 747324309504|
|SOBNZDC12A6D4FC103|b80344d063b5ccb32...|    1|240518168576|1382979469312|
|SOBXALG12A8C13C108|b80344d063b5ccb32...|    1|240518168576|1288490188800|
|SOBYHAJ12A6701BF1D|b80344d063b5ccb32...|    1|240518168576|1176821039104|
+------------------+--------------------+-----+------------+-------------+



Py4JJavaError: An error occurred while calling o3017.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 605.0 failed 1 times, most recent failure: Lost task 0.0 in stage 605.0 (TID 7119, localhost, executor driver): org.apache.spark.SparkException: Failed to execute user defined function($anonfun$1: (double) => int)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIterator.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anonfun$8$$anon$1.hasNext(WholeStageCodegenExec.scala:377)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:408)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:408)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:408)
	at scala.collection.Iterator$$anon$12.hasNext(Iterator.scala:439)
	at scala.collection.Iterator$JoinIterator.hasNext(Iterator

In [77]:
#read file and parse data
#train
#model
#make recommendation

In [78]:
lines = sc.textFile("/Users/samanthapace/Desktop/SongData/train_triplets.txt")
parts = lines.map(lambda l: array([int(x) for x in l.split("\t")]))


In [81]:
rank=1
numIterations=1
model = ALS.trainImplicit(parts, rank, numIterations, alpha = 0.01).fit()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 22.0 failed 1 times, most recent failure: Lost task 0.0 in stage 22.0 (TID 26, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/Users/samanthapace/spark-2.1.0-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 174, in main
    process()
  File "/Users/samanthapace/spark-2.1.0-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 169, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/Users/samanthapace/spark-2.1.0-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/serializers.py", line 268, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/Users/samanthapace/spark-2.1.0-bin-hadoop2.7/python/pyspark/rdd.py", line 1339, in takeUpToNumLeft
    yield next(iterator)
  File "<ip

In [ ]:
rawPredictions = model.transform(playCountRDD)
predictions = rawPredictions\
    .withColumn("playCount", rawPredictions.rating.cast("double"))\
    .withColumn("prediction", rawPredictions.prediction.cast("double"))
evaluator =\
    RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

In [30]:
training = sc.textFile("/Users/samanthapace/Desktop/SongData/train_triplets.txt").map(parse_playCount).cache()
test = sc.textFile("/Users/samanthapace/Desktop/SongData/train_triplets.txt").map(parse_playCount)

test.load_playCount()

model = ALS.train(test, rank = 10, iterations = 5)

predictions = model.predictAll(test.map(lambda x: (x[0], x[1])))



Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 9.0 failed 1 times, most recent failure: Lost task 0.0 in stage 9.0 (TID 13, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/Users/samanthapace/spark-2.1.0-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 174, in main
    process()
  File "/Users/samanthapace/spark-2.1.0-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 169, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/Users/samanthapace/spark-2.1.0-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/serializers.py", line 268, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/Users/samanthapace/spark-2.1.0-bin-hadoop2.7/python/pyspark/rdd.py", line 1339, in takeUpToNumLeft
    yield next(iterator)
  File "<ipyt